In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np

np.set_printoptions(precision=3)
np.random.seed(317042)

from timeit import default_timer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# error reporting function
def create_error_dict(**kwargs):
	error_dict = {}
	
	for err_name, errs in kwargs.items():
		mean = np.mean(errs)
		max = np.max(errs)

		error_dict[f"{err_name}_mean"] = mean
		error_dict[f"{err_name}_max"] = max

	return error_dict

### Hessenbergification

In [3]:
from scipy.linalg import hessenberg
from utils import get_random_symmetric_matrix, make_upper_hessenberg

def test_hessenberg(test_count = 100, size = 5, a = 0.0, b = 1.0):
	times, gt_times = [], []
	
	for t in range(test_count):
		A = get_random_symmetric_matrix(size, a, b)

		start = default_timer()
		make_upper_hessenberg(A)
		end = default_timer()

		times.append(end - start)

		start = default_timer()
		hessenberg(A)
		end = default_timer()

		gt_times.append(end - start)


	return times, gt_times

In [6]:
times, gt_times = test_hessenberg(100, 200)
create_error_dict(ours=times, scipy=gt_times)

{'ours_mean': np.float64(0.022182323001907207),
 'ours_max': np.float64(0.03100240000640042),
 'scipy_mean': np.float64(0.02487172900058795),
 'scipy_max': np.float64(0.29665879998356104)}

### Hessenberg QR

In [7]:
from utils import hessenberg_qr

def test_hessenberg_qr(test_count = 100, size = 5, a = 0.0, b = 1.0):
	times, gt_times = [], []

	for t in range(test_count):
		A = get_random_symmetric_matrix(size, a, b)

		H, _ = make_upper_hessenberg(A)
		
		start = default_timer()
		hessenberg_qr(H)
		end = default_timer()

		times.append(end - start)

		start = default_timer()
		np.linalg.qr(H)
		end = default_timer()

		gt_times.append(end - start)

	return times, gt_times


In [8]:
times, gt_times = test_hessenberg_qr(100, 50)
create_error_dict(hessenberg_qr=times, generic_qr=gt_times)

{'hessenberg_qr_mean': np.float64(0.001287365999596659),
 'hessenberg_qr_max': np.float64(0.0023849999997764826),
 'generic_qr_mean': np.float64(0.00015175699722021817),
 'generic_qr_max': np.float64(0.00040599997737444937)}

In [9]:
times, gt_times = test_hessenberg_qr(100, 100)
create_error_dict(hessenberg_qr=times, generic_qr=gt_times)

{'hessenberg_qr_mean': np.float64(0.002619778997905087),
 'hessenberg_qr_max': np.float64(0.00598270000773482),
 'generic_qr_mean': np.float64(0.002375703998550307),
 'generic_qr_max': np.float64(0.003678499982925132)}

In [10]:
times, gt_times = test_hessenberg_qr(100, 200)
create_error_dict(hessenberg_qr=times, generic_qr=gt_times)

{'hessenberg_qr_mean': np.float64(0.005427810001710895),
 'hessenberg_qr_max': np.float64(0.006075200013583526),
 'generic_qr_mean': np.float64(0.007556002998317126),
 'generic_qr_max': np.float64(0.008458299998892471)}

In [11]:
times, gt_times = test_hessenberg_qr(100, 400)
create_error_dict(hessenberg_qr=times, generic_qr=gt_times)

{'hessenberg_qr_mean': np.float64(0.012431529000168666),
 'hessenberg_qr_max': np.float64(0.019479299982776865),
 'generic_qr_mean': np.float64(0.025459886999742595),
 'generic_qr_max': np.float64(0.0333229000098072)}

In [12]:
times, gt_times = test_hessenberg_qr(100, 800)
create_error_dict(hessenberg_qr=times, generic_qr=gt_times)

{'hessenberg_qr_mean': np.float64(0.029923908999189733),
 'hessenberg_qr_max': np.float64(0.045188100019004196),
 'generic_qr_mean': np.float64(0.09073173800104996),
 'generic_qr_max': np.float64(0.1270496000070125)}

### Practical-QR Step

In [15]:
from eigen import practical_qr

In [16]:
def test_qr_iteration(test_count = 100, size = 5, a = 0.0, b = 1.0):
	times, gt_times = [], []

	for t in range(test_count):
		A = get_random_symmetric_matrix(size, a, b)
		
		start = default_timer()
		practical_qr(A)
		end = default_timer()

		times.append(end - start)

		start = default_timer()
		np.linalg.eig(A)
		end = default_timer()

		gt_times.append(end - start)

		
	return times, gt_times

In [18]:
times, gt_times = test_qr_iteration(5, 50)
create_error_dict(ours=times, numpy=gt_times)

{'ours_mean': np.float64(0.09570241999463178),
 'ours_max': np.float64(0.10612000001128763),
 'numpy_mean': np.float64(0.0006238200003281236),
 'numpy_max': np.float64(0.0006505000055767596)}

In [19]:
times, gt_times = test_qr_iteration(5, 100)
create_error_dict(ours=times, numpy=gt_times)

{'ours_mean': np.float64(0.408254500007024),
 'ours_max': np.float64(0.4275903000088874),
 'numpy_mean': np.float64(0.008335179998539388),
 'numpy_max': np.float64(0.014173800009302795)}

In [20]:
times, gt_times = test_qr_iteration(5, 500)
create_error_dict(ours=times, numpy=gt_times)

{'ours_mean': np.float64(14.508027259993833),
 'ours_max': np.float64(15.22019769999315),
 'numpy_mean': np.float64(0.1918787800008431),
 'numpy_max': np.float64(0.21329169999808073)}

### SVD

In [22]:
from svd import svd

In [23]:
def test_svd(test_count = 100, size = 5, a = 0.0, b = 1.0, square=False):
	times, gt_times = [], []
	
	for t in range(test_count):
		m = size
		n = size

		if not square:
			n = np.random.randint(1, size+1)

			if np.random.uniform() <= 0.5:
				m, n = n, m

		A = np.random.uniform(a, b, (m, n))

		start = default_timer()
		svd(A)
		end = default_timer()

		times.append(end - start)

		start = default_timer()
		np.linalg.svd(A, full_matrices=False)
		end = default_timer()

		gt_times.append(end - start)

	return times, gt_times

In [26]:
times, gt_times = test_svd(5, 50)
create_error_dict(ours=times, numpy=gt_times)

{'ours_mean': np.float64(0.022426299995277076),
 'ours_max': np.float64(0.08493459998862818),
 'numpy_mean': np.float64(0.00043311999179422854),
 'numpy_max': np.float64(0.0016158999933395535)}

In [27]:
times, gt_times = test_svd(5, 100)
create_error_dict(ours=times, numpy=gt_times)

{'ours_mean': np.float64(0.10645417999476195),
 'ours_max': np.float64(0.2473269999900367),
 'numpy_mean': np.float64(0.001976139994803816),
 'numpy_max': np.float64(0.004693399998359382)}

In [28]:
times, gt_times = test_svd(5, 500)
create_error_dict(ours=times, numpy=gt_times)

{'ours_mean': np.float64(6.577215020003496),
 'ours_max': np.float64(11.247191800008295),
 'numpy_mean': np.float64(0.06905266001122072),
 'numpy_max': np.float64(0.11733060001279227)}